In [11]:
# imports

import uproot
import numpy as np
import pandas
import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use([hep.style.CMS])


In [12]:
Ntuple = uproot.open("NTuple.root")
tree = Ntuple["mytuple"]

In [13]:
tree.keys()

['j_Px', 'j_Py', 'j_Pz', 'j_E', 'j_eta', 'j_phi']

In [17]:
DF = tree.arrays(library = "pd")